# Load a Python plugin

It is assumed this Notebook is running in an environment with OpenAssetIO and OpenAssetIO-MediaCreation installed into it, such that libraries and headers are trivially discoverable (e.g. Conda environment described in the README).

#### Load Python library

Load the Python available in the Conda environment this Notebook is running in.

Use xeus-cling pragma extension to add search paths and load the library.

In [ ]:
#pragma cling add_library_path("$CONDA_PREFIX/lib")

In [ ]:
#pragma cling load("python3.11")

#### Load OpenAssetIO library

In [ ]:
#pragma cling load("openassetio-python")

#### Bootstrap Python

In [ ]:
#include <python3.11/Python.h>
Py_Initialize();

## Define a logger

In [ ]:
#include <iostream>

#include <openassetio/log/LoggerInterface.hpp>

struct Logger : openassetio::log::LoggerInterface {
  void log(Severity severity, const openassetio::Str& message) override {
      std::cout << message << "\n";
  }
};

auto logger = std::make_shared<Logger>();

## Query Python plugins

In [ ]:
#include <memory>

#include <openassetio/hostApi/ManagerImplementationFactoryInterface.hpp>
#include <openassetio/python/hostApi.hpp>

const auto factory = openassetio::python::hostApi::createPythonPluginSystemManagerImplementationFactory(logger);
const openassetio::Identifiers identifiers = factory->identifiers();

identifiers

## Define host interface

In [ ]:
#include <openassetio/hostApi/HostInterface.hpp>

struct Host : openassetio::hostApi::HostInterface {
  openassetio::Identifier identifier() const { return "org.demo"; }
  openassetio::Str displayName() const { return "Demo Host"; }
  openassetio::InfoDictionary info() const { return {}; }
};

const auto host = std::make_shared<Host>();

## Get a manager instance

Here we pass a path to the `config.toml`, we could also use the `OPENASSETIO_DEFAULT_CONFIG` environment variable.

In [ ]:
#include <openassetio/hostApi/ManagerFactory.hpp>

const auto manager = openassetio::hostApi::ManagerFactory::defaultManagerForInterface("resources/config.toml", host, factory, logger)

## Resolve some data

In [ ]:
#include <openassetio/hostApi/Manager.hpp>

#include <openassetio_mediacreation/traits/content.hpp>

using openassetio_mediacreation::traits::content::LocatableContentTrait;

const openassetio::ContextPtr context = manager->createContext();

const openassetio::EntityReference ref = manager->createEntityReference("bal:///demo_project/logos/openassetio/latest");

const auto data = manager->resolve(ref, {LocatableContentTrait::kId}, context);

std::string url = LocatableContentTrait(data).getLocation();

url

In [ ]:
auto file_path = url.substr(7);
file_path

#### (boilerplate to render images in the notebook)

This next snippet is from the xeus-cling docs, required in order to display images in the Notebook: https://xeus-cling.readthedocs.io/en/latest/rich_display.html

In [ ]:
/**
 * Modified from: https://xeus-cling.readthedocs.io/en/latest/rich_display.html
 * 
 * Why isn't this built-in ???
 */ 

#include <string>
#include <fstream>

#include "xtl/xbase64.hpp"
#include "nlohmann/json.hpp"

namespace nl = nlohmann;

namespace im
{
    struct jpg
    {
        inline jpg(const std::string& filename)
        {
            std::ifstream fin(filename, std::ios::binary);
            m_buffer << fin.rdbuf();
        }

        std::stringstream m_buffer;
    };

    nl::json mime_bundle_repr(const jpg& i)
    {
        auto bundle = nl::json::object();
        bundle["image/jpeg"] = xtl::base64encode(i.m_buffer.str());
        return bundle;
    }
}

#### Render the resolved and retrieved data

In [ ]:
im::jpg logo{file_path};
logo